In [1]:
import tsfresh
import pandas
import os
import numpy as np
from imageio import imread
from tqdm import tqdm

In [2]:
rgb_matrix = imread('frames/train/stable/stable_1.png')
nb_time = len(rgb_matrix)
nb_colonnes_dataframe = len(rgb_matrix[0])


In [13]:
col_features = ['f'+str(i) for i in range(nb_colonnes_dataframe)]
col_names =  ['id', 'time'] + col_features

In [9]:
images = []
y = {}

paths = ['frames/train/malfunction/', 'frames/train/stable/']
counter = 0
for i in range(len(paths)):
    files = [f for f in os.listdir(paths[i]) if f.lower().endswith(".png")]
    for file in files:
        images.append(os.path.join(paths[i], file))
        y[counter] = i
        counter += 1

In [16]:
my_df  = pandas.DataFrame(columns = col_names)

for i in tqdm(range(len(images))): 
    image = images[i]
    rgb_matrix = imread(image)
    df_image = pandas.DataFrame(rgb_matrix[:, :, 0], columns = col_features)
    df_image['time'] = range(nb_time)
    df_image['id'] = i
    my_df = my_df.append(df_image)
    
my_df = my_df[my_df.columns[::-1]]



  0%|          | 0/198 [00:00<?, ?it/s]

  3%|▎         | 6/198 [00:00<00:04, 45.45it/s]

  5%|▍         | 9/198 [00:00<00:05, 33.11it/s]

  6%|▌         | 11/198 [00:00<00:07, 25.16it/s]

  7%|▋         | 13/198 [00:00<00:09, 19.93it/s]

  8%|▊         | 15/198 [00:00<00:10, 16.70it/s]

  9%|▊         | 17/198 [00:00<00:12, 14.37it/s]

 10%|▉         | 19/198 [00:01<00:14, 12.69it/s]

 11%|█         | 21/198 [00:01<00:15, 11.26it/s]

 12%|█▏        | 23/198 [00:01<00:17, 10.14it/s]

 13%|█▎        | 25/198 [00:01<00:18,  9.14it/s]

 13%|█▎        | 26/198 [00:01<00:20,  8.43it/s]

 14%|█▎        | 27/198 [00:02<00:21,  7.92it/s]

 14%|█▍        | 28/198 [00:02<00:22,  7.43it/s]

 15%|█▍        | 29/198 [00:02<00:23,  7.06it/s]

 15%|█▌        | 30/198 [00:02<00:24,  6.77it/s]

 16%|█▌        | 31/198 [00:02<00:25,  6.53it/s]

 16%|█▌        | 32/198 [00:02<00:26,  6.30it/s]

 17%|█▋        | 33/198 [00:03<00:26,  6.11it/s]

 17%|█▋        | 34/198 [00:03<00:27,  5.89it/s]

 18%|█▊  

 81%|████████  | 160/198 [01:08<00:31,  1.21it/s]

 81%|████████▏ | 161/198 [01:09<00:31,  1.19it/s]

 82%|████████▏ | 162/198 [01:09<00:30,  1.17it/s]

 82%|████████▏ | 163/198 [01:10<00:29,  1.17it/s]

 83%|████████▎ | 164/198 [01:11<00:29,  1.17it/s]

 83%|████████▎ | 165/198 [01:12<00:28,  1.17it/s]

 84%|████████▍ | 166/198 [01:13<00:27,  1.16it/s]

 84%|████████▍ | 167/198 [01:14<00:26,  1.15it/s]

 85%|████████▍ | 168/198 [01:15<00:26,  1.15it/s]

 85%|████████▌ | 169/198 [01:16<00:25,  1.14it/s]

 86%|████████▌ | 170/198 [01:16<00:24,  1.14it/s]

 86%|████████▋ | 171/198 [01:17<00:23,  1.13it/s]

 87%|████████▋ | 172/198 [01:18<00:23,  1.13it/s]

 87%|████████▋ | 173/198 [01:19<00:22,  1.12it/s]

 88%|████████▊ | 174/198 [01:20<00:21,  1.11it/s]

 88%|████████▊ | 175/198 [01:21<00:20,  1.11it/s]

 89%|████████▉ | 176/198 [01:22<00:20,  1.10it/s]

 89%|████████▉ | 177/198 [01:23<00:19,  1.09it/s]

 90%|████████▉ | 178/198 [01:24<00:18,  1.09it/s]

 90%|█████████ | 179/198 [01:25

In [18]:
my_df

,time,id,f99,f98,f97,f96,f95,f94,f93,f92,...,f106,f105,f104,f103,f102,f101,f100,f10,f1,f0
0,0,0,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
1,1,0,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
2,2,0,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
3,3,0,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
4,4,0,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,475,197,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
476,476,197,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
477,477,197,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255
478,478,197,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,255,255


In [23]:
my_df.astype('float32').dtypes

time    float32
id      float32
f99     float32
f98     float32
f97     float32
         ...   
f101    float32
f100    float32
f10     float32
f1      float32
f0      float32
Length: 642, dtype: object

In [24]:
features_filtered_direct = tsfresh.extract_relevant_features(my_df.astype('float32'), pandas.Series(y),
                                                     column_id='id', column_sort='time')




Feature Extraction:   0%|          | 0/15 [00:00<?, ?it/s]


Feature Extraction:   7%|▋         | 1/15 [49:29<11:32:47, 2969.09s/it]


Feature Extraction:  13%|█▎        | 2/15 [49:46<7:31:26, 2083.60s/it] 


Feature Extraction:  20%|██        | 3/15 [49:50<4:51:56, 1459.71s/it]

KeyboardInterrupt: 